### Train a word2vec model using Keras
[reference link](https://www.tensorflow.org/tutorials/text/word2vec)

In [1]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
%load_ext tensorboard
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [3]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [4]:
path_to_file = r"center.txt"
# Use the non empty lines to construct a tf.data.TextLineDataset object for the next step
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [5]:
# lowercase the text and remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 8192
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [6]:
# Call TextVectorization.adapt on the text dataset to create vocabulary
vectorize_layer.adapt(text_ds.batch(1024))

In [7]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'to', 'of', 'a', 'and', 'in', 'on', 'said', 'that', 'for', 'by', 'trump', 'he', 'with', 'u', 's', 'is', 'as']


In [8]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [9]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

12160


In [10]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[  13    1  279   11 1353  210    7    1 2574 1088] => ['trump', '[UNK]', 'putin', 'for', 'holding', 'back', 'in', '[UNK]', 'spy', 'dispute']
[2170  940  225    1    7    2  200 1180   91   12] => ['russians', 'leave', 'country', '[UNK]', 'in', 'the', 'us', 'ordered', 'out', 'by']
[  33 4050  516 3503    1   15    1   19    5   33] => ['new', 'york’s', 'times', 'square', '[UNK]', 'with', '[UNK]', 'as', 'a', 'new']
[  13    1    1  141 1439 6901    7    1 1503  299] => ['trump', '[UNK]', '[UNK]', 'party', 'shows', 'rift', 'in', '[UNK]', 'movement', 'supporters']
[  13 1113  379  200  374    8  151  127   27    1] => ['trump', 'aide', 'says', 'us', 'sanctions', 'on', 'russia', 'may', 'be', '[UNK]']


In [11]:
# sequences is now a list of int encoded sentences. 
# Just call the generate_training_data function defined earlier to generate training examples for the word2vec model. 
# To recap, the function iterates over each word from each sequence to collect positive and negative context words. 
# Length of target, contexts and labels should be the same, representing the total number of training examples.
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 12160/12160 [01:15<00:00, 160.25it/s]




targets.shape: (80480,)
contexts.shape: (80480, 5)
labels.shape: (80480, 5)


In [12]:
# To perform efficient batching for the potentially large number of training examples, use the tf.data.Dataset API. 
# After this step, you would have a tf.data.Dataset object of (target_word, context_word), (label) elements to train your word2vec model
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [13]:
num_ns = 4
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [17]:
# Compile the model with the tf.keras.optimizers.Adam optimizer
embedding_dim = 512
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [18]:
# define a callback to log training statistics for Tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [19]:
# Train the model on the dataset for some number of epochs:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
78/78 [==============================] - 1s 13ms/step - loss: 1.5731 - accuracy: 0.5071
Epoch 2/20
78/78 [==============================] - 1s 9ms/step - loss: 1.3163 - accuracy: 0.7590
Epoch 3/20
78/78 [==============================] - 1s 10ms/step - loss: 0.9640 - accuracy: 0.7776
Epoch 4/20
78/78 [==============================] - 1s 11ms/step - loss: 0.7072 - accuracy: 0.8354
Epoch 5/20
78/78 [==============================] - 1s 10ms/step - loss: 0.5318 - accuracy: 0.8794
Epoch 6/20
78/78 [==============================] - 1s 9ms/step - loss: 0.4111 - accuracy: 0.9109
Epoch 7/20
78/78 [==============================] - 1s 8ms/step - loss: 0.3266 - accuracy: 0.9339
Epoch 8/20
78/78 [==============================] - 1s 8ms/step - loss: 0.2661 - accuracy: 0.9501
Epoch 9/20
78/78 [==============================] - 1s 8ms/step - loss: 0.2218 - accuracy: 0.9607
Epoch 10/20
78/78 [==============================] - 1s 8ms/step - loss: 0.1888 - accuracy: 0.9681
Epoch 11/20
78/

In [20]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [21]:
vocab.index('trump')

13

In [22]:
weights[vocab.index('trump')]

array([ 0.03844438, -0.14008585, -0.21221514, -0.13628022, -0.18974096,
       -0.20457222, -0.05379911,  0.1412965 , -0.06501113,  0.00121054,
       -0.23720323,  0.00704947,  0.276176  ,  0.11510568,  0.15288785,
        0.10557481, -0.4820325 , -0.11222402, -0.06627972,  0.23284675,
        0.14378582, -0.00586037,  0.1329638 ,  0.13589063, -0.09969267,
       -0.08916178,  0.14804177,  0.00980355,  0.16982275, -0.24848534,
       -0.12184989,  0.02265077,  0.15964778, -0.00547457,  0.11001627,
       -0.01093417,  0.1286592 ,  0.03448049, -0.26520285,  0.2564958 ,
       -0.0454016 ,  0.09357715, -0.26502272, -0.07577291, -0.142107  ,
       -0.2193175 , -0.35863206,  0.07468792, -0.05995445,  0.12344135,
       -0.2715645 , -0.00840358, -0.14407666,  0.29216278, -0.1547988 ,
       -0.07570764,  0.16539103, -0.15211287,  0.20302172,  0.20943414,
       -0.2039899 ,  0.1466455 , -0.01837552, -0.00768824,  0.06770298,
       -0.08434218,  0.01423713, -0.14368957, -0.14679132, -0.05

In [23]:
# create and save the vectors and metadata files
out_v = io.open('center_vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('center_metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()